In [ ]:
VOCABS_DIR = '../data/interim/'
REFERENCE_VOCAB_PATH = VOCABS_DIR + 'reference_vocab.pkl'
TRANSLATION_VOCAB_PATH = VOCABS_DIR + 'translation_vocab.pkl'
MODELS_DIR = '../models/'
SEQ2SEQ_LSTMS_PATH = MODELS_DIR + "seq2seq - lstms.pth"
SEQ2SEQ_GRUS_PATH = MODELS_DIR + "seq2seq - grus.pth"

In [ ]:
# Load the entire model
model = torch.load('model.pth')

# Load only the model state dictionary
# First, instantiate your model architecture
model = Seq2Seq(encoder, decoder, device)
# Then load the state dictionary
model.load_state_dict(torch.load('model_state_dict.pth'))


In [ ]:
def detoxify_sentence(sentence, src_vocab, trg_vocab, model, device, max_len=50):
    model.eval()
    
    # Tokenize the sentence, add the <sos> and <eos> tokens, and numericalize
    tokens = [src_vocab.get(token, src_vocab[PAD_TOKEN]) for token in sentence.split()]
    numericalized_tokens = [src_vocab[SOS_TOKEN]] + tokens + [src_vocab[EOS_TOKEN]]
    
    # Convert to Tensor and add a batch dimension
    src_tensor = torch.LongTensor(numericalized_tokens).unsqueeze(0).to(device)
    
    # Predict the target sequence
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
        trg_indexes = [trg_vocab[SOS_TOKEN]]

        for _ in range(max_len):
            trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
            with torch.no_grad():
                output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
            
            # Get the predicted next token (the one with the highest probability)
            pred_token = output.argmax(1).item()
            trg_indexes.append(pred_token)

            # If the <eos> token is predicted, stop
            if pred_token == trg_vocab[EOS_TOKEN]:
                break
    
    # Convert the predicted numerical tokens to words
    trg_tokens = [list(trg_vocab.keys())[list(trg_vocab.values()).index(idx)] for idx in trg_indexes]
    
    # Return the words after the <sos> token
    return trg_tokens[1:-1]

# Load vocabularies
with open(REFERENCE_VOCAB_PATH, 'rb') as f:
    src_vocab = pickle.load(f)
with open(TRANSLATION_VOCAB_PATH, 'rb') as f:
    trg_vocab = pickle.load(f)




In [ ]:
# Change this sentence if you want to make another prediction
src_sentence = "I'm not the fucking best guy in this whole shitty universe"
detoxified_tokens = detoxify_sentence(src_sentence, src_vocab, trg_vocab, model, device)
print(" ".join(detoxified_tokens))